In [1]:
#Import Libraries
import pandas as pd
import numpy as np
# Loading Data
df = pd.read_csv('/Users/AribM/Desktop/Mutual Funds India-Data Analysis/SourceData.csv')
# Filtering and Cleaning Data
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
df=df[df['Date'] >= '2018-07-23'] # Analysing Data for 5 year period, i.e, 2018-07-23 to 2023-07-23

In [2]:
df['min_date'] = df.groupby(by='Scheme_Code')['Date'].transform('min') # Getting the earliest date of fund data
df['max_date'] = df.groupby(by='Scheme_Code')['Date'].transform('max') # Getting the latest date of fund data

In [3]:
df = df.sort_values(by=['Scheme_Code','Date']).reset_index(drop=True) # Sorting by Scheme_Code and Date

In [4]:
df=df[df['max_date'] >= '2023-01-01'] #Removing Mutual Funds whose data has not been recieved after 2022
df['no_of_days']=(df['max_date']-df['min_date']).dt.days # Calculating the number of days of data for the given fund
df=df[df['no_of_days'] >= 365] # Filtering out MFs whose data is available for less than 1 year

In [5]:
df = df[df['NAV'] != 0] # Removing 0 NAV values
df.dropna(subset='NAV',inplace=True) # Removing NaN NAV values for consistency and precision 

In [6]:
df['returns'] = df.groupby('Scheme_Code')['NAV'].pct_change() * 100  # Getting the daily returns of the Funds
df['returns'] = df['returns'].fillna(0) # Filling NaN values in returns column with 0 
df['std_dev'] = df.groupby('Scheme_Code')['returns'].transform('std') # Calculating fund wise Standard Deviation 


In [7]:
df['average_fund_return'] = df.groupby('Scheme_Code')['returns'].transform('mean') * 252 # Annualizing the fund returns

In [8]:
# Computing Values for calculating total Active period between the analysis period
df['scheme_iv'] = df.groupby('Scheme_Code')['NAV'].transform('first')
df['scheme_fv'] = df.groupby('Scheme_Code')['NAV'].transform('last')
df_active_years = df.groupby('Scheme_Code').agg(
    scheme_min_date=('min_date', 'min'),
    scheme_max_date=('max_date', 'max')
).reset_index()

df_active_years['active_years'] = (df_active_years['scheme_max_date'] - df_active_years['scheme_min_date']).dt.days / 365
# Merge back with the original DataFrame
df = pd.merge(df,df_active_years,on='Scheme_Code', how='left')
df = df[(df['scheme_iv'] > 0) & (df['active_years'] > 0)] # Prevent division by 0
df['CAGR'] = ((df['scheme_fv']/df['scheme_iv']) ** (1/df['active_years']) - 1 )* 100 # Calculating Fund wise CAGR

In [9]:
df.drop(['scheme_fv','scheme_iv'],axis=1,inplace=True) # Dropping final and initial NAV values of funds from main dataframe

In [13]:
# Calculating Sortino Ratio

# assuming the risk free rate of 4%
risk_free_rate = 4

def calculate_sortino(group):
    scheme_avg_return = group['returns'].mean() * 252 # Annualizing returns 
    downside_returns = group[group['returns'] < risk_free_rate]['returns'] - risk_free_rate # computing downside returns
    downside_dev = downside_returns.std() * np.sqrt(252) # Calculating downside deviation of funds and annualizing 

    if downside_dev == 0 or np.isnan(downside_dev): # if no negative values then return NaN
        return np.nan
    
    return (scheme_avg_return - risk_free_rate) / downside_dev # Sortino formula

df_sortino = df.groupby('Scheme_Code',group_keys=False).apply(calculate_sortino,include_groups=False).reset_index(name="Sortino_Ratio")
# Created a dataframe containing sortino values 

In [14]:
df = df.merge(df_sortino,on = 'Scheme_Code',how='left') # Merging sortino df to main df

In [15]:
# Calculating Sharpe Ratio
rfr = 0.04 # assuming 4% annual risk free rate
df_sharpe = df.groupby('Scheme_Code')['returns'].agg(['mean','std']).reset_index()
df_sharpe['mean_annual'] = df_sharpe['mean'] * 252
df_sharpe['std_annual'] = df_sharpe['std'] * (252 ** 0.5)

df_sharpe['Sharpe_annual'] = (df_sharpe['mean_annual'] - rfr) / df_sharpe['std_annual'] # Annualizing Sharpe values
df_sharpe.fillna({'Sharpe_annual': 0}, inplace=True)  # replacing Nan values by 0
df = df.merge(df_sharpe[['Scheme_Code','Sharpe_annual']],on='Scheme_Code',how='left') # merge back to main df

In [16]:
# Creating a shorter dataframe with relevant data and all calculated fields
df_processed = df.drop(['scheme_min_date','scheme_max_date','min_date','max_date','Date','NAV','no_of_days','returns','average_fund_return'],axis = 1).copy()
df_processed['active_years'] = df_processed['active_years'].round()
cols_to_round = ['std_dev','CAGR']
df_processed[cols_to_round] = df_processed[cols_to_round].round(3) # rounding off column values upto 3 decimals
df_processed.drop_duplicates(subset='Scheme_Code',inplace=True) # Only 1 row per scheme
df_processed = df_processed.reset_index()
df_processed.drop(columns=['index'],inplace=True)
df_processed.index.name = "S.No" # Renaming Index Name

In [17]:
# Dropping Nan values if any:
df_processed.dropna(subset='CAGR',inplace=True)
print(df_processed) # printing to check the new dataframe

                             Fund_House          Scheme_Type  \
S.No                                                           
0     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
1     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
2     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
3     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
4     Aditya Birla Sun Life Mutual Fund  360 ONE Mutual Fund   
...                                 ...                  ...   
6198  Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
6199            Mirae Asset Mutual Fund   Open Ended Schemes   
6200            Mirae Asset Mutual Fund   Open Ended Schemes   
6201            Mirae Asset Mutual Fund   Open Ended Schemes   
6202            Mirae Asset Mutual Fund   Open Ended Schemes   

                                 Scheme_Category  Scheme_Code  \
S.No                                                            
0           Equity Scheme - Large & M

In [18]:
with pd.ExcelWriter("processed_data.xlsx", engine="openpyxl") as writer:
    df_processed.to_excel(writer, sheet_name="Processed_Data", index=False)

In [19]:
df.drop(['scheme_min_date','scheme_max_date'],axis=1,inplace=True)
df.fillna({'returns':0},inplace=True)
df.to_csv("detailed_data.csv",index=False)